In [59]:
from lxml import etree

In [60]:

l_path = ["/home/ziane212/projects/web_HT/flask_test/uploads/HT/1190_kings_england.xml",
          "/home/ziane212/projects/web_HT/flask_test/uploads/HT/1487_cronicque.xml",
          "/home/ziane212/projects/web_HT/flask_test/uploads/HT/1862_Histoire_de_la_Normandie_ancienne.xml"]                             

path_table = "/home/ziane212/Téléchargements/subj_pre.tsv"

In [61]:
def make_form(w):
    form = ""
    if len(list(w))!=0:
        if list(w)[0].tag == 'choice':
            form = list(w)[0][1].text
        else:
            for child in w.iter():
                if child.text:
                    form += child.text
                if child.tail:
                    form += child.tail
    else:
        form = w.text
    
    return form

def extract_SUBJnoDET(path_in):
    tree = etree.parse(open(path_in, encoding='utf-8'))
    root = tree.getroot()

    list_results = []

    filename = path_in.split('/')[len(path_in.split('/'))-1]

    for book in root.findall('.//div[@type="book"]'):
        book_nb = book.get('n')
        book_ch = int(book_nb)
        for chapter in book.findall('.//div[@type="chapter"]'):
            chapter_nb = chapter.get('n')
            chapter_ch = int(chapter_nb)
            for section in chapter.findall('.//div[@type="section"]'):
                section_nb = section.get('n')
                section_ch = int(section_nb)
                for para in section.findall('.//p'):
                    para_nb = para.get('n')
                    para_ch = int(para_nb)

                    for id_, s in enumerate(para.findall('.//s')):
                        sentence_nb = s.get('n')
                        sentence_ch = int(sentence_nb)

                        sent_id = book_nb+'_'+chapter_nb+'_'+section_nb+'_'+para_nb+'_'+sentence_nb

                        for w in s.findall('.//w'):        
                            if w.get('udpos') == 'VERB':
                                id_gov = w.get('n')

                                var = False

                                for w2 in para.findall('.//s')[id_]:

                                    id_dep = w2.get('n')

                                    if w2.get('head') == id_gov and w2.get('function') == 'nsubj' and id_dep < id_gov:
                                        var = True

                                    # elif w2.get('head') == id_gov and id_dep < id_gov and w2.get('function') == 'nummod':
                                    #     var = True

                                    # elif w2.get('head') == id_gov and w2.get('udpos') == 'PROPN' and w2.get('function') == 'flat':
                                    #     var = True

                                    # elif w2.get('head') == id_gov and id_dep > id_gov and w2.get('function') == 'appos':
                                    #     var = True

                                if var == True:
                                    sent = []

                                    l_context = []
                                    r_context = []
                                    for w3 in s.findall('.//w')[:int(id_dep)-1]:
                                        form = make_form(w3)

                                        l_context.append(form)

                                    for w3 in s.findall('.//w')[int(id_dep):]:
                                        form = make_form(w3)

                                        r_context.append(form)

                                    sent.append(filename)
                                    sent.append(sent_id)
                                    cut_l_context = (len(l_context)-1)-9
                                    sent.append(' '.join(l_context[cut_l_context:len(l_context)]))
                                    sent.append(s.findall('.//w')[int(id_dep)-1].text)
                                    sent.append(' '.join(r_context[:9]))
                                    #dev print(sent)
                                    list_results.append(sent)

    return list_results


         

In [62]:
for path_in in l_path:
    
    list_results = extract_SUBJnoDET(path_in)
    
    with open(path_table, 'a', encoding="utf-8") as tsv:
        for i in list_results:
            # print(i)
            
            tsv.write('\t'.join(i)+'\n')

In [63]:
a


NameError: name 'a' is not defined

list_sub = ['acl', 'acl:relcl', 'advcl', 'ccomp', 'xcomp']

def extract_noSUBJ(path_in):
    tree = etree.parse(open(path_in, encoding='utf-8'))
    root = tree.getroot()

    list_results = []

    filename = path_in.split('/')[len(path_in.split('/'))-1]

    for book in root.findall('.//div[@type="book"]'):
        book_nb = book.get('n')
        book_ch = int(book_nb)
        for chapter in book.findall('.//div[@type="chapter"]'):
            chapter_nb = chapter.get('n')
            chapter_ch = int(chapter_nb)
            for section in chapter.findall('.//div[@type="section"]'):
                section_nb = section.get('n')
                section_ch = int(section_nb)
                for para in section.findall('.//p'):
                    para_nb = para.get('n')
                    para_ch = int(para_nb)

                    for id_, s in enumerate(para.findall('.//s')):
                        sentence_nb = s.get('n')
                        sentence_ch = int(sentence_nb)

                        sent_id = book_nb+'_'+chapter_nb+'_'+section_nb+'_'+para_nb+'_'+sentence_nb

                        for w in s.findall('.//w'):        
                            if w.get('udpos') == 'VERB':
                                id_gov = w.get('n')
                                head_gov = w.get('head')
                                var = False

                                for w2 in para.findall('.//s')[id_]:

                                    id_dep = w2.get('n')

                                    if w2.get('head') == id_gov and (w2.get('function') == 'nsubj' or w2.get('function') == 'expl'):
                                        var = True
                                        
                                    elif w.get('function') == 'conj':
                                        
                                        for w3 in para.findall('.//s')[id_]:

                                            if head_gov == w3.get('head') and w3.get('function') == 'nsubj':
                                                var = True
                                                
                                if w.get('function') == 'root':
                                    type_s = 'principale'
                                elif w.get('function') in list_sub:
                                    type_s = 'subordonnée_'+w.get('function')
                                        

                                if var == False:
                                    sent = []

                                    l_context = []
                                    r_context = []
                                    for w3 in s.findall('.//w')[:int(id_gov)-1]:
                                        form = make_form(w3)

                                        l_context.append(form)

                                    for w3 in s.findall('.//w')[int(id_gov):]:
                                        form = make_form(w3)

                                        r_context.append(form)

                                    sent.append(filename)
                                    sent.append(sent_id)
                                    cut_l_context = (len(l_context)-1)-9
                                    sent.append(' '.join(l_context[cut_l_context:len(l_context)]))
                                    sent.append(s.findall('.//w')[int(id_gov)-1].text)
                                    sent.append(' '.join(r_context[:9]))
                                    sent.append('Nul')
                                    sent.append(type_s)
                                    #dev print(sent)
                                    list_results.append(sent)
                                    
                                else:
                                    sent = []

                                    l_context = []
                                    r_context = []
                                    for w3 in s.findall('.//w')[:int(id_gov)-1]:
                                        form = make_form(w3)

                                        l_context.append(form)

                                    for w3 in s.findall('.//w')[int(id_gov):]:
                                        form = make_form(w3)

                                        r_context.append(form)

                                    sent.append(filename)
                                    sent.append(sent_id)
                                    cut_l_context = (len(l_context)-1)-9
                                    sent.append(' '.join(l_context[cut_l_context:len(l_context)]))
                                    sent.append(s.findall('.//w')[int(id_gov)-1].text)
                                    sent.append(' '.join(r_context[:9]))
                                    sent.append(type_s)
                                    #dev print(sent)
                                    list_results.append(sent)
                                    

    return list_results